# Support Vector Machines

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
import matplotlib as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from tqdm import tqdm
from statistics import mean
import warnings
import math
from statistics import mean
from sklearn.svm import SVC
from timeit import default_timer as timer
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score, accuracy_score

In [2]:
hp = pd.read_csv('HP_OHE_3class.csv')
hp = hp.drop('Unnamed: 0',axis=1)

In [3]:
def getXandY(df):
    df.drop(df.tail(20).index,inplace=True) 
    x = df.drop(['HP_Forbidden_clean'],axis=1)
    y = df.HP_Forbidden_clean
    return(x,y)

def trainTest(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True)
    return(X_train, X_test, y_train, y_test)

smote = SMOTE(random_state=42)
X,Y=getXandY(hp)
X_train, X_test, y_train, y_test = trainTest(X,Y)
X_train_Smote, y_train_Smote= smote.fit_resample(X_train, y_train)

mrmrO = ['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek', 'temperature', 'humidity', 
         'pressure', 'heavy intensity rain', 'light rain', 'broken clouds', 'scattered clouds', 
         'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain', 'heavy intensity rain', 
         'mist', 'scattered clouds']
X_train_Smot_r = X_train_Smote[mrmrO]
X_test_r = X_test[mrmrO]

print(sorted(Counter(y_train_Smote).items()))

[(1.0, 5590), (2.0, 5590), (3.0, 5590)]


In [8]:
lsvm = LinearSVC(C=1, loss="hinge")
lsvm.fit(X_train_Smot_r,y_train_Smote)
y_pred=lsvm.predict(X_test_r)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.63      0.73      0.68      2345
         2.0       0.51      0.49      0.50      1535
         3.0       0.33      0.21      0.26       845

    accuracy                           0.56      4725
   macro avg       0.49      0.48      0.48      4725
weighted avg       0.54      0.56      0.54      4725



/Users/rodrigoledesmagarces/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


# Testing out different variable cambinations according to feature selection methods

Important functions

In [4]:
#Function to perform oversampling
def overSampling(X_train, y_train, y_test, method):
    X_train_os, y_train_os= method.fit_resample(X_train, y_train)
    # Check the number of records after over sampling
    #print(sorted(Counter(y_train_os).items())) 
    return(X_train_os, y_train_os)

In [5]:
smote = SMOTE(random_state=42)

def testModel(df,var_order,n_vars,n_loops):
    for j in tqdm(range(1,n_vars)):
        #split our dataframe into X and Y
        x,y=getXandY(df)
        #create the lists to store metrics
        acc = []
        rec = []
        preci = []
        f1 = []
        for i in range(n_loops):
            #split the dataFrame into test and train
            X_train, X_test, y_train, y_test = trainTest(x,y)
            #Oversample the train dataset with SMOTE
            X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
            #define the variables order 
            X_train_os_r = X_train_os[var_order]
            X_test_r = X_test[var_order]
            df1= X_train_os_r.iloc[:, 0:j] #use only part of the variables
            lsvm = LinearSVC(C=1, loss="hinge")
            lsvm.fit(df1, y_train_os).decision_function(X_test_r.iloc[:, 0:j])
            #lg = LogisticRegression(max_iter=20000)
            #lg.fit(df1, y_train_os).decision_function(X_test_r.iloc[:, 0:j])
            y_pred=lsvm.predict(X_test_r.iloc[:, 0:j])
            ac=metrics.accuracy_score(y_test, y_pred)
            acc.append(ac)
            p=metrics.precision_score(y_test, y_pred,average='macro')
            preci.append(p)
            r=metrics.recall_score(y_test, y_pred,average='macro')
            rec.append(r)
            f=metrics.f1_score(y_test, y_pred, average='macro')
            f1.append(f)
        print(df1.columns)
        print("For {} features: \n Accuracy: {} \n Precision: {} \n Recall: {} \n F1 score: {}".format(
        j,mean(acc),mean(preci),mean(rec),mean(f1)))
        print(classification_report(y_test, y_pred))

In [6]:
def analizeDF(df,order,n_vars,n_loops):
    for i in range(len(order)):
        print('------------------------- Analyzing method {} -------------------------'.format(method[i]))
        print('The variable order is: \n {}'.format(order[i]))
        testModel(df,order[i],n_vars,n_loops)
        print('\n \n')

### One Hot Encoding

In [9]:
#Variable order in one hot encoding:
pear_corrO = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'fog',	'overcast clouds',	'heavy intensity rain',	'minute',	'haze',	'thunderstorm with light rain',	'scattered clouds',	'clear sky',	'mist',	'light intensity drizzle',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'year',	'light rain',	'humidity',	'hour',	'Pandemic']
kend_corrO = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'heavy intensity rain',	'fog',	'year',	'overcast clouds',	'minute',	'haze',	'thunderstorm with light rain',	'mist',	'light intensity drizzle',	'scattered clouds',	'clear sky',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'light rain',	'humidity',	'hour',	'Pandemic']
mutInf_classO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_regO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_class2O = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek',	'year',	'holiday',	'shower rain',	'light rain',	'thunderstorm',	'fog',	'broken clouds',	'Pandemic',	'thunderstorm with rain',	'light intensity drizzle',	'thunderstorm with light rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds',	'clear sky']
varThreO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',]
mrmrO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds']

orderOHE=[pear_corrO,kend_corrO,mutInf_classO,mutInf_regO,mutInf_class2O,varThreO,mrmrO]

method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'varianceThreshold','MRMR']

In [16]:
import warnings
warnings.filterwarnings('ignore')

hp_oHe = hp
analizeDF(hp_oHe,orderOHE,28,n_loops=20)

  0%|          | 0/27 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'shower rain', 'broken clouds', 'fog', 'overcast clouds', 'heavy intensity rain', 'minute', 'haze', 'thunderstorm with light rain', 'scattered clouds', 'clear sky', 'mist', 'light intensity drizzle', 'few clouds', 'thunderstorm', 'very heavy rain', 'moderate rain', 'thunderstorm with rain', 'year', 'light rain', 'humidity', 'hour', 'Pandemic']


  4%|▎         | 1/27 [00:04<02:04,  4.81s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.46427333766515055 
 Precision: 0.294389307619276 
 Recall: 0.3675773647619927 
 F1 score: 0.31317531958385614
              precision    recall  f1-score   support

         1.0       0.57      0.72      0.64      2397
         2.0       0.00      0.00      0.00      1410
         3.0       0.20      0.39      0.26       810

    accuracy                           0.44      4617
   macro avg       0.26      0.37      0.30      4617
weighted avg       0.33      0.44      0.38      4617



  7%|▋         | 2/27 [00:09<01:58,  4.74s/it]

...

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute'],
      dtype='object')
For 25 features: 
 Accuracy: 0.5669808626106827 
 Precision: 0.4624242482489552 
 Recall: 0.47813151105067875 
 F1 score: 0.44176707632751644
              precision    recall  f1-score   support

         1.0       0.69      0.77      0.73      1990
         2.0       0.36      0.35      0.35       866
         3.0       0.38      0.26      0.31       645

    accuracy                           0.57      3501
   macro avg       0.48      0.46      0.46      3501
weighted avg       0.55      0.57      0.56      3501



 96%|█████████▋| 26/27 [01:58<00:07,  7.11s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze'],
      dtype='object')
For 26 features: 
 Accuracy: 0.5623032904148784 
 Precision: 0.45642457989344976 
 Recall: 0.47499747157845634 
 F1 score: 0.4418079525599455
              precision    recall  f1-score   support

         1.0       0.72      0.75      0.73      2034
         2.0       0.33      0.09      0.14       878
         3.0       0.31      0.61      0.41       583

    accuracy                           0.56      3495
   macro avg       0.45      0.48      0.43      3495
weighted avg       0.55      0.56      0.53      3495



100%|██████████| 27/27 [02:04<00:00,  4.60s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze', 'few clouds'],
      dtype='object')
For 27 features: 
 Accuracy: 0.5601175121811407 
 Precision: 0.45509983899187767 
 Recall: 0.4749113355158412 
 F1 score: 0.42965586140676887
              precision    recall  f1-score   support

         1.0       0.72      0.75      0.73      2047
         2.0       0.21      0.01      0.02       831
         3.0       0.31      0.66      0.42       611

    accuracy                           0.56      3489
   macro avg       0.41      0.47      0.39      3489
weighted avg       0.52      0.56      0.51      3489


 



### Manual Encoding

In [25]:
hp_ME = pd.read_csv("harryPotterClean.csv")
hp_ME.drop(hp_ME.tail(17).index,inplace=True) 
hp_ME.isnull().sum()

Unnamed: 0                        0
month                             0
day                               0
year                              0
hour                              0
minute                            0
holiday                           0
day.1                             0
Pandemic                          0
temperature                       0
humidity                          0
pressure                          0
report                            0
Harry_Potter_and_the_Forbidden    0
dtype: int64

In [27]:
#Replace times ending in 5 and also compacting the rest to have only 6 classes
b=hp_ME.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df3=pd.DataFrame(b)
df3.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis3=pd.concat([hp_ME, df3], axis=1)
hp3 = hp_bis3.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp3 = hp3.drop('Unnamed: 0',axis=1)
hp3.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)
hp3 = hp3[hp3.HP_Forbidden_clean != 0] #delete rows with 0 min
hp3.HP_Forbidden_clean.unique()


array([2., 1., 3.])

In [29]:
hp3.head()

,month,day,year,hour,minute,holiday,dayOfTheWeek,Pandemic,temperature,humidity,pressure,report,HP_Forbidden_clean
0,0.636364,0.966667,0.0,0.608696,0.4,0.0,0.0,0.0,0.721791,0.740741,0.2,0.6250,2.0
1,0.636364,0.966667,0.0,0.608696,0.6,0.0,0.0,0.0,0.721791,0.740741,0.2,0.6250,2.0
2,0.636364,0.966667,0.0,0.608696,0.8,0.0,0.0,0.0,0.711940,0.679012,0.2,0.4375,2.0
3,0.636364,0.966667,0.0,0.608696,1.0,0.0,0.0,0.0,0.723881,0.679012,0.2,0.3750,2.0
4,0.636364,0.966667,0.0,0.652174,0.0,0.0,0.0,0.0,0.724179,0.679012,0.2,0.3750,1.0


In [34]:
HP_ME = hp3

pear_corr = ['temperature','holiday','day','month','pressure','dayOfTheWeek',
            'report','minute','year','humidity','hour',	'Pandemic']
kend_corr = ['temperature','holiday','month','day','pressure','dayOfTheWeek','report','year',
             'minute','humidity','hour','Pandemic']
mutInf_class = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_reg = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_class2 = ['day','temperature','month','humidity','hour','dayOfTheWeek','pressure',
                 'holiday','year','report','minute','Pandemic']
varThre = ['month','day','year','hour','minute','holiday','dayOfTheWeek',
           'temperature','humidity','pressure','report']
mrmr = ['temperature','dayOfTheWeek','hour','holiday','humidity','day',
        'year','month','pressure','minute','report']
method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'variableThreshold','MRMR']

order=[pear_corr,kend_corr,mutInf_class,mutInf_reg,mutInf_class2,varThre,mrmr]

In [35]:
analizeDF(HP_ME,order,13,n_loops=20)

  0%|          | 0/12 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  8%|▊         | 1/12 [00:01<00:18,  1.66s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.45713074803983894 
 Precision: 0.3033649889156127 
 Recall: 0.3704077590608607 
 F1 score: 0.315164008852292
              precision    recall  f1-score   support

         1.0       0.47      0.08      0.14      2357
         2.0       0.31      0.85      0.45      1529
         3.0       0.16      0.02      0.04       833

    accuracy                           0.32      4719
   macro avg       0.31      0.32      0.21      4719
weighted avg       0.36      0.32      0.22      4719



 17%|█▋        | 2/12 [00:03<00:16,  1.63s/it]

...

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.56202456886369 
 Precision: 0.49143791440848966 
 Recall: 0.4806048329720324 
 F1 score: 0.4783854559367139
              precision    recall  f1-score   support

         1.0       0.68      0.74      0.71      2281
         2.0       0.47      0.42      0.44      1202
         3.0       0.31      0.27      0.29       750

    accuracy                           0.57      4233
   macro avg       0.48      0.48      0.48      4233
weighted avg       0.55      0.57      0.56      4233



 92%|█████████▏| 11/12 [00:19<00:01,  1.90s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.5589070262597587 
 Precision: 0.48638662034096203 
 Recall: 0.47470837642555896 
 F1 score: 0.47416255338053587
              precision    recall  f1-score   support

         1.0       0.68      0.73      0.71      2264
         2.0       0.44      0.46      0.45      1209
         3.0       0.30      0.23      0.26       754

    accuracy                           0.56      4227
   macro avg       0.48      0.47      0.47      4227
weighted avg       0.55      0.56      0.55      4227



100%|██████████| 12/12 [00:21<00:00,  1.76s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 12 features: 
 Accuracy: 0.5589433783463634 
 Precision: 0.4914859415180935 
 Recall: 0.4786012041308916 
 F1 score: 0.4757588650842991
              precision    recall  f1-score   support

         1.0       0.65      0.75      0.70      2228
         2.0       0.43      0.43      0.43      1224
         3.0       0.34      0.20      0.25       769

    accuracy                           0.55      4221
   macro avg       0.47      0.46      0.46      4221
weighted avg       0.53      0.55      0.54      4221


 



### Ordinal Encoding

In [37]:
hp_OE = pd.read_csv("harryPotterCleanOE.csv")
hp_OE.drop(hp_OE.tail(17).index,inplace=True) 
hp_OE = hp_OE.drop('Unnamed: 0',axis=1)
hp_OE.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)
hp_OE.isnull().sum()

month                             0
day                               0
year                              0
hour                              0
minute                            0
holiday                           0
dayOfTheWeek                      0
Pandemic                          0
temperature                       0
humidity                          0
pressure                          0
report                            0
Harry_Potter_and_the_Forbidden    0
dtype: int64

In [40]:
#Replace times ending in 5 and also compacting the rest to have only 6 classes
a=hp_OE.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df1=pd.DataFrame(a)
df1.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis=pd.concat([hp_OE, df1], axis=1)
hp2 = hp_bis.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp2 = hp2[hp2.HP_Forbidden_clean != 0] #delete rows with 0 min
hp2.HP_Forbidden_clean.unique()

array([2., 1., 3.])

In [43]:
#Variable order in ordinal encoding:
pear_corrOE = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'report',	'minute',	'year',	'humidity',	'hour',	'Pandemic']
kend_corrOE = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'report',	'year',	'minute',	'humidity',	'hour',	'Pandemic',]
mutInf_classOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_regOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_class2OE = ['day',	'temperature',	'month',	'humidity',	'hour',	'dayOfTheWeek',	'pressure',	'holiday',	'year',	'report',	'minute',	'Pandemic',]
varThreOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'report']
mrmrOE = ['temperature',	'dayOfTheWeek',	'hour',	'holiday',	'humidity',	'day',	'year',	'month',	'pressure',	'minute',	'report']

orderOE=[pear_corrOE,kend_corrOE,mutInf_classOE,mutInf_regOE,mutInf_class2OE,varThreOE,mrmrOE]

In [44]:
analizeDF(hp2,orderOE,13,n_loops=20)

  0%|          | 0/12 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  8%|▊         | 1/12 [00:01<00:18,  1.70s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.4449353676626404 
 Precision: 0.2963079136801052 
 Recall: 0.36387036500620584 
 F1 score: 0.3093639800119144
              precision    recall  f1-score   support

         1.0       0.53      0.86      0.66      2430
         2.0       0.42      0.23      0.30      1486
         3.0       0.00      0.00      0.00       803

    accuracy                           0.51      4719
   macro avg       0.32      0.36      0.32      4719
weighted avg       0.41      0.51      0.43      4719



 17%|█▋        | 2/12 [00:03<00:16,  1.66s/it]

...

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.5584691708008505 
 Precision: 0.488055431178006 
 Recall: 0.47709243727319706 
 F1 score: 0.47575111074628995
              precision    recall  f1-score   support

         1.0       0.66      0.75      0.70      2233
         2.0       0.55      0.26      0.35      1249
         3.0       0.32      0.46      0.38       751

    accuracy                           0.56      4233
   macro avg       0.51      0.49      0.48      4233
weighted avg       0.57      0.56      0.54      4233



 92%|█████████▏| 11/12 [00:19<00:01,  1.92s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.5619943222143364 
 Precision: 0.4864992167133188 
 Recall: 0.4743427657803073 
 F1 score: 0.4750222490552903
              precision    recall  f1-score   support

         1.0       0.66      0.76      0.71      2237
         2.0       0.47      0.43      0.45      1250
         3.0       0.35      0.25      0.29       740

    accuracy                           0.57      4227
   macro avg       0.49      0.48      0.48      4227
weighted avg       0.55      0.57      0.56      4227



100%|██████████| 12/12 [00:21<00:00,  1.78s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 12 features: 
 Accuracy: 0.5598081023454158 
 Precision: 0.4876458426754078 
 Recall: 0.4756471552245021 
 F1 score: 0.4748345723960861
              precision    recall  f1-score   support

         1.0       0.66      0.74      0.70      2239
         2.0       0.42      0.45      0.43      1230
         3.0       0.37      0.20      0.26       752

    accuracy                           0.56      4221
   macro avg       0.48      0.46      0.46      4221
weighted avg       0.54      0.56      0.54      4221


 



# Hyperparameter Tuning

In [74]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 0.5, 1, 5, 10, 50, 100], 'loss':['hinge', 'squared_hinge']}

grid = GridSearchCV(LinearSVC(),param_grid)
grid.fit(X_train_Smot_r,y_train_Smote)

GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': [0.1, 0.5, 1, 5, 10, 50, 100],
                         'loss': ['hinge', 'squared_hinge']})

In [75]:
print(grid.best_estimator_)

LinearSVC(C=1)


### Trying out the results

In [11]:
c = [0.1, 0.5, 1, 5, 10, 50, 100]
LOSS = ["squared_hinge", "hinge"]
loops = 20
arr = []

mu = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek',	'year',
      'holiday',	'shower rain',	'light rain']

for i in range(len(c)):
    for j in range(len(LOSS)):
        for k in range(loops):
            X_train, X_test, y_train, y_test = trainTest(X,Y)
            #Oversample the train dataset with SMOTE
            X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
            #define the variables order 
            X_train_os_r = X_train_os[mu]
            X_test_r = X_test[mu]
            lsvm = LinearSVC(C=c[i], loss=LOSS[j])
            lsvm.fit(X_train_os_r,y_train_os)
            y_pred=lsvm.predict(X_test_r)
            score = accuracy_score(y_test, lsvm.predict(X_test_r))
            arr.append(score)
        print("For the parameters are C:{}, and loss:{} accuracy is:".format(c[i],LOSS[j]))
        print(mean(arr))

For the parameters are C:0.1, and loss:squared_hinge accuracy is:
0.5477989417989418
For the parameters are C:0.1, and loss:hinge accuracy is:
0.553962962962963
For the parameters are C:0.5, and loss:squared_hinge accuracy is:
0.5508536155202822
For the parameters are C:0.5, and loss:hinge accuracy is:
0.5534259259259259
For the parameters are C:1, and loss:squared_hinge accuracy is:
0.5517544973544973
For the parameters are C:1, and loss:hinge accuracy is:
0.5533192239858906
For the parameters are C:5, and loss:squared_hinge accuracy is:
0.5526228269085411
For the parameters are C:5, and loss:hinge accuracy is:
0.5536534391534391
For the parameters are C:10, and loss:squared_hinge accuracy is:
0.5527854203409759
For the parameters are C:10, and loss:hinge accuracy is:
0.5534338624338624
For the parameters are C:50, and loss:squared_hinge accuracy is:
0.5516084656084657
For the parameters are C:50, and loss:hinge accuracy is:
0.5514964726631393
For the parameters are C:100, and loss:sq

# Polynomial SVM

In [4]:
from sklearn.svm import SVC

In [85]:
psvm = SVC(kernel="poly", degree=2, coef0=1, C=5)
psvm.fit(X_train_Smot_r,y_train_Smote)
y_pred=psvm.predict(X_test_r)
score = accuracy_score(y_test, psvm.predict(X_test_r))
score

0.662010582010582

In [7]:
from sklearn.svm import SVC
from timeit import default_timer as timer
from datetime import timedelta

c=[0.01,0.1,1,10,50]
loops = 3

mu = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek',	'year',
      'holiday',	'shower rain',	'light rain']

def polySMV(X,Y,mu,deg,c,loops):
    arr = []
    highest=0
    for i in range(len(deg)):
        for k in range(len(c)):
            for l in range(loops):
                start = timer()
                X_train, X_test, y_train, y_test = trainTest(X,Y)
                #Oversample the train dataset with SMOTE
                X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
                #define the variables order 
                X_train_os_r = X_train_os[mu]
                X_test_r = X_test[mu]
                psvm = SVC(kernel="poly", degree=deg[i], coef0=1, C=c[k])
                psvm.fit(X_train_os_r,y_train_os)
                y_pred=psvm.predict(X_test_r)
                score = accuracy_score(y_test, psvm.predict(X_test_r))
                arr.append(score)
                print("-")
                # End timer
                end1 = timer()
                #print(timedelta(seconds=end1-start))
                #check the best configuration
                if score > highest:
                    highest = score
                    description = "best values = degree:{}, and c:{}".format(deg[i],c[k])
                else:
                    score = score
            print("For the parameters degree:{}, and c:{} accuracy is:".format(deg[i],c[k]))
            print(mean(arr))
            arr = []
    print(description)

In [10]:
degg = [4]
polySMV(X,Y,mu,degg,c,1)

-
For the parameters degree:4, and c:0.01 accuracy is:
0.6573544973544974
-
For the parameters degree:4, and c:0.1 accuracy is:
0.7085714285714285
-
For the parameters degree:4, and c:1 accuracy is:
0.7462433862433863
-
For the parameters degree:4, and c:10 accuracy is:
0.7635978835978836
-
For the parameters degree:4, and c:100 accuracy is:
0.7936507936507936
best values = degree:4, and c:100


In [ ]:
degg = [5]
polySMV(X,Y,mu,degg,c,1)

-
For the parameters degree:5, and c:0.01 accuracy is:
0.6931216931216931
-
For the parameters degree:5, and c:0.1 accuracy is:
0.7443386243386243
-
For the parameters degree:5, and c:1 accuracy is:
0.771005291005291
-
For the parameters degree:5, and c:10 accuracy is:
0.7978835978835979


In [8]:
degg = [6]
polySMV(X,Y,mu,degg,c,1)

-
For the parameters degree:6, and c:0.01 accuracy is:
0.7354497354497355
-
For the parameters degree:6, and c:0.1 accuracy is:
0.7724867724867724
-
For the parameters degree:6, and c:1 accuracy is:
0.7817989417989418
-
For the parameters degree:6, and c:10 accuracy is:
0.8084656084656084
-
For the parameters degree:6, and c:50 accuracy is:
0.8239153439153439
best values = degree:6, and c:50


In [9]:
degg = [7]
polySMV(X,Y,mu,degg,c,1)

-
For the parameters degree:7, and c:0.01 accuracy is:
0.76
-
For the parameters degree:7, and c:0.1 accuracy is:
0.7822222222222223
-
For the parameters degree:7, and c:1 accuracy is:
0.7974603174603174
-
For the parameters degree:7, and c:10 accuracy is:
0.813968253968254
-
For the parameters degree:7, and c:50 accuracy is:
0.8344973544973545
best values = degree:7, and c:50


# RBF kernel on SVM

In [86]:
rsvm = SVC(kernel="rbf", gamma=5, C=0.001)
rsvm.fit(X_train_Smot_r,y_train_Smote)
y_pred_r=rsvm.predict(X_test_r)
scorer = accuracy_score(y_test, rsvm.predict(X_test_r))

In [87]:
scorer

0.6014814814814815

In [7]:
gam = [1, 0.1, 0.01, 0.001, 0.0001]
c=[0.1, 1, 5, 10, 50, 100, 1000]
loops = 10
arr = []
highest=0

mu = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek',	'year',
      'holiday',	'shower rain',	'light rain']

for i in range(len(gam)):
    for j in range(len(c)):
        for l in range(loops):
            start = timer()
            X_train, X_test, y_train, y_test = trainTest(X,Y)
            #Oversample the train dataset with SMOTE
            X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
            #define the variables order 
            X_train_os_r = X_train_os[mu]
            X_test_r = X_test[mu]
            rsvm = SVC(kernel="rbf", gamma=gam[i], C=c[j])
            rsvm.fit(X_train_os_r,y_train_os)
            y_pred=rsvm.predict(X_test_r)
            score = accuracy_score(y_test, rsvm.predict(X_test_r))
            arr.append(score)
            # End timer
            end1 = timer()
            #print(timedelta(seconds=end1-start))
            #check the best configuration
            if score > highest:
                highest = score
                description = "best values = gamma:{}, and c:{}".format(gam[i],c[j])
            else:
                score = score
        print("For the parameters gamma:{}, and C:{} accuracy is:".format(gam[i],c[j]))
        print(mean(arr))
        arr = []
print(description)

For the parameters gamma:1, and C:0.1 accuracy is:
0.6379047619047619
For the parameters gamma:1, and C:1 accuracy is:
0.7084867724867725
For the parameters gamma:1, and C:5 accuracy is:
0.7429206349206349
For the parameters gamma:1, and C:10 accuracy is:
0.7542857142857143
For the parameters gamma:1, and C:50 accuracy is:
0.7808465608465609
For the parameters gamma:1, and C:100 accuracy is:
0.7887619047619048
For the parameters gamma:1, and C:1000 accuracy is:
0.8131640211640212
For the parameters gamma:0.1, and C:0.1 accuracy is:
0.5446772486772486
For the parameters gamma:0.1, and C:1 accuracy is:
0.6142433862433863
For the parameters gamma:0.1, and C:5 accuracy is:
0.6504126984126984
For the parameters gamma:0.1, and C:10 accuracy is:
0.6582645502645502
For the parameters gamma:0.1, and C:50 accuracy is:
0.6834920634920635
For the parameters gamma:0.1, and C:100 accuracy is:
0.6958095238095238
For the parameters gamma:0.1, and C:1000 accuracy is:
0.7304338624338624
For the paramete